# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv exported from WeatherPy 
WeatherPy_file = '../WeatherPy/city_weather_sample.csv'
WeatherPy_df = pd.read_csv(WeatherPy_file,index_col=0)
WeatherPy_df

,Latitude,Longitude,City,Country,Time,Temp (Celsius),Max Temp (Celsius),Cloudiness %,Wind Speed (m/s),Humidity %
0,-42.88,147.33,Hobart,AU,1583862019,9.20,11.00,0,4.10,87
1,19.22,-104.70,San Patricio,MX,1583862202,29.00,29.00,20,3.10,70
2,-34.00,24.73,Kruisfontein,ZA,1583862202,18.64,18.64,100,6.56,92
3,-34.53,20.04,Bredasdorp,ZA,1583862202,20.82,21.00,67,6.20,56
4,-38.00,-57.56,Mar del Plata,AR,1583862202,19.47,20.00,40,6.20,68
...,...,...,...,...,...,...,...,...,...,...
566,-22.43,-151.33,Moerai,PF,1583862264,26.92,26.92,100,1.44,69
567,71.04,27.85,Mehamn,NO,1583862264,-5.00,-5.00,75,9.30,57
568,29.03,21.55,Jalu,LY,1583862038,17.95,17.95,0,5.03,29
569,50.68,47.45,Piterka,RU,1583862264,7.60,7.60,98,5.57,46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Configure gmaps and display heatmap based on the retrieved data from OpenWeather API
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(WeatherPy_df[['Latitude','Longitude']],weights=WeatherPy_df['Humidity %']
                                    ,max_intensity=100,point_radius=20,opacity=0.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Make new DataFrame based on warm weather conditions in the Western Hemisphere
warm_df = WeatherPy_df.loc[(WeatherPy_df['Max Temp (Celsius)'] < 25) & (WeatherPy_df['Max Temp (Celsius)'] > 15),:] 
minimal_wind_df = warm_df.loc[warm_df['Wind Speed (m/s)'] < 5,:]
not_cloudy_df = minimal_wind_df.loc[minimal_wind_df['Cloudiness %'] < 15,:]

west_hemisphere_ideal_weather_df = not_cloudy_df.loc[not_cloudy_df['Longitude'] < 0,:]
hotel_df = west_hemisphere_ideal_weather_df.copy()
hotel_df.reset_index(inplace=True)
hotel_df.drop(columns='index',inplace=True)
hotel_df

,Latitude,Longitude,City,Country,Time,Temp (Celsius),Max Temp (Celsius),Cloudiness %,Wind Speed (m/s),Humidity %
0,-29.34,-49.73,Torres,BR,1583862208,24.94,24.94,4,4.67,77
1,-30.18,-50.21,Cidreira,BR,1583862213,24.61,24.61,14,4.75,85
2,-29.75,-50.01,Capão da Canoa,BR,1583862216,24.61,24.61,14,4.75,85
3,-33.04,-71.63,Valparaíso,CL,1583862222,18.98,20.00,0,2.60,77
4,29.58,-9.50,Tiznit Province,MA,1583862223,21.83,21.83,0,2.97,24
5,43.67,-7.41,Cervo,ES,1583862229,16.63,17.22,0,0.89,77
6,8.95,-82.12,Chiriquí Grande,PA,1583862233,23.33,23.33,11,4.47,44
7,40.16,-8.33,Lata,PT,1583862238,16.12,17.22,0,4.60,72
8,38.79,-121.24,Rocklin,US,1583862250,14.57,17.78,1,4.10,71
9,38.70,-9.42,Cascais,PT,1583862251,21.07,24.00,0,4.10,64


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Call google places API to retrieve the hotel closest to the city coordinates and add the hotel name to the dataframe
hotel_names = []
hotel_coordinates = []
for x in range(len(hotel_df)):
    lat = hotel_df['Latitude'][x]
    lng = hotel_df['Longitude'][x]
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&locationbias=circle:2000@{lat},{lng}&fields=geometry/location,name&key={g_key}'
    response_json = requests.get(url).json()
    hotel_names.append(response_json['candidates'][0]['name'])
    hotel_coordinates.append((response_json['candidates'][0]['geometry']['location']['lat'],response_json['candidates'][0]['geometry']['location']['lng']))

hotel_df['Hotel Names'] = hotel_names
hotel_df

,Latitude,Longitude,City,Country,Time,Temp (Celsius),Max Temp (Celsius),Cloudiness %,Wind Speed (m/s),Humidity %,Hotel Names,"Hotel Coordinates (Lat,Lng)"
0,-29.34,-49.73,Torres,BR,1583862208,24.94,24.94,4,4.67,77,Encantos Torres Hotel,"(-29.3401208, -49.7271933)"
1,-30.18,-50.21,Cidreira,BR,1583862213,24.61,24.61,14,4.75,85,Hotel Sulmar,"(-29.9870843, -50.1391303)"
2,-29.75,-50.01,Capão da Canoa,BR,1583862216,24.61,24.61,14,4.75,85,Hotel Napoli,"(-29.768353, -50.0232327)"
3,-33.04,-71.63,Valparaíso,CL,1583862222,18.98,20.00,0,2.60,77,WineBox Valparaiso,"(-33.0509196, -71.61860469999999)"
4,29.58,-9.50,Tiznit Province,MA,1583862223,21.83,21.83,0,2.97,24,Riad Janoub,"(29.6988672, -9.7226731)"
5,43.67,-7.41,Cervo,ES,1583862229,16.63,17.22,0,0.89,77,Hotel Ego,"(43.6857298, -7.5804931)"
6,8.95,-82.12,Chiriquí Grande,PA,1583862233,23.33,23.33,11,4.47,44,Loma Partida Aqua Lodge Boom Shiva,"(9.1490102, -82.1755506)"
7,40.16,-8.33,Lata,PT,1583862238,16.12,17.22,0,4.60,72,QUINTA DAS LÁGRIMAS,"(40.197625, -8.432693)"
8,38.79,-121.24,Rocklin,US,1583862250,14.57,17.78,1,4.10,71,DoubleTree Suites by Hilton Hotel Sacramento -...,"(38.6083031, -121.26697)"
9,38.70,-9.42,Cascais,PT,1583862251,21.07,24.00,0,4.10,64,Hotel Cascais Miragem Health & Spa,"(38.702343, -9.4110014)"


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(hotel_coordinates,info_box_content=hotel_info)
fig.add_layer(hotel_markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))